In [ ]:
import os
import warnings
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_forecasting
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [ ]:
df = pd.read_csv("/home/aimlproject1/home/imp_mean.csv")

In [ ]:
df.head()

In [ ]:
df = df.sort_values(by=['investment_id','time_id'])
df.head()


In [ ]:
df = df.astype({'time_id': np.int16})
df = df.astype({'investment_id': np.int16})
df =df.drop(columns=['Unnamed: 0'], axis=0)
x = df['investment_id'].unique()
df.head()
# lst = [1415, 2800, 3662, 85, 2558, 905, 1364, 72, 890, 709]
# print(df.shape)
# for i in lst:
#    df.drop(df.index[df['investment_id'] == i], inplace=True)   
# print(df.shape)

In [ ]:

# for i in x:
#    small_df = df.loc[df['investment_id'] == i]
#    small_df.drop_duplicates(subset ="time_id",
#                      keep = 'first', inplace = True)

#    print(i)
#    column_names = [str(s) for s in small_df.columns]
#    print(small_df.shape)
#    train = TimeSeriesDataSet(small_df,
#                         categorical_encoders={"investment_id": pytorch_forecasting.data.encoders.NaNLabelEncoder().fit(small_df['investment_id'])},
#                          group_ids = ['investment_id'],
#                             time_idx="time_id",
#                          target='target',
#                          time_varying_unknown_reals=column_names,
#                           min_encoder_length=1,  # keep encoder length long (as it is in the validation set)
#                     max_encoder_length=100,
#                          )
#    # break

In [ ]:
column_names = ['a','b','c','d','e','f','g','h','i','j']
train = TimeSeriesDataSet(df,
                        categorical_encoders={"investment_id": pytorch_forecasting.data.encoders.NaNLabelEncoder().fit(df['investment_id'])},
                         group_ids = ['investment_id'],
                            time_idx="time_id",
                         target='target',
                         time_varying_unknown_reals=column_names,
                          min_encoder_length=1,  # keep encoder length long (as it is in the validation set)
                    max_encoder_length=100,
                         )

In [ ]:
validation = TimeSeriesDataSet.from_dataset(train, df, predict=True, stop_randomization=True)

In [ ]:
batch_size = 128  # set this between 32 to 128
train_dataloader = train.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)


In [ ]:
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()


In [ ]:
# pl.seed_everything(42)
# trainer = pl.Trainer(
#     gpus=1,
#     # clipping gradients is a hyperparameter and important to prevent divergance
#     # of the gradient for recurrent neural networks
#     gradient_clip_val=0.1,
# )


# tft = TemporalFusionTransformer.from_dataset(
#     train,
#     # not meaningful for finding the learning rate but otherwise very important
#     learning_rate=0.03,
#     hidden_size=16,  # most important hyperparameter apart from learning rate
#     # number of attention heads. Set to up to 4 for large datasets
#     attention_head_size=1,
#     dropout=0.1,  # between 0.1 and 0.3 are good values
#     hidden_continuous_size=8,  # set to <= hidden_size
#     output_size=7,  # 7 quantiles by default
#     loss=QuantileLoss(),
#     # reduce learning rate if no improvement in validation loss after x epochs
#     reduce_on_plateau_patience=4,
# )

In [ ]:
# res = trainer.tuner.lr_find(
#     tft,
#     train_dataloaders=train_dataloader,
#     val_dataloaders=val_dataloader,
#     max_lr=10.0,
#     min_lr=1e-6,
# )

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=5,
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    # callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    train,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    # log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
import tensorflow as tf
with tf.device("gpu"):
    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

In [ ]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)


In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)
(actuals - predictions).abs().mean()
